# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

# Import Data

In [10]:
df = pd.read_csv('../cleaned_data/merged.csv')
df.tail()

,fips_code,state,estimated_hesitant,estimated_hesitant_or_unsure,estimated_strongly_hesitant,social_vulnerability_index_(svi),svi_category,cvac_level_of_concern_for_vaccination_rollout,cvac_level_of_concern,percent_adults_fully_vaccinated_against_covid-19_(as_of_6/10/21),...,series_complete_18pluspop_pct_ur_equity,series_complete_65pluspop_pct_ur_equity,administered_dose1_recip_5plus,administered_dose1_recip_5pluspop_pct,series_complete_5plus,series_complete_5pluspop_pct,pop,state_pop,state_pop_pct,distributed
3111,55079,WISCONSIN,0.1505,0.2144,0.0880,0.81,Very High Vulnerability,0.30,Low Concern,0.543,...,4.0,4.0,628736.0,71.3,563558.0,63.9,945591.0,5852490,16.157072,1598161.0
3112,55121,WISCONSIN,0.1611,0.2126,0.0959,0.28,Low Vulnerability,0.31,Low Concern,0.612,...,8.0,8.0,19889.0,72.2,18474.0,67.1,29653.0,5852490,0.506673,50117.0
3113,56001,WYOMING,0.1949,0.2673,0.1405,0.25,Low Vulnerability,0.63,High Concern,0.445,...,8.0,8.0,22819.0,61.5,19926.0,53.7,38844.0,581075,6.684851,56638.0
3114,55067,WISCONSIN,0.1518,0.2007,0.0906,0.35,Low Vulnerability,0.19,Very Low Concern,0.481,...,8.0,8.0,10423.0,57.1,9871.0,54.1,19204.0,5852490,0.328134,32457.0
3115,55099,WISCONSIN,0.1528,0.2039,0.0906,0.16,Very Low Vulnerability,0.20,Very Low Concern,0.542,...,8.0,8.0,8390.0,65.7,7906.0,61.9,13355.0,5852490,0.228193,22572.0


In [5]:
df.columns

Index(['fips_code', 'state', 'estimated_hesitant',
       'estimated_hesitant_or_unsure', 'estimated_strongly_hesitant',
       'social_vulnerability_index_(svi)', 'svi_category',
       'cvac_level_of_concern_for_vaccination_rollout',
       'cvac_level_of_concern',
       'percent_adults_fully_vaccinated_against_covid-19_(as_of_6/10/21)',
       'percent_hispanic',
       'percent_non-hispanic_american_indian/alaska_native',
       'percent_non-hispanic_asian', 'percent_non-hispanic_black',
       'percent_non-hispanic_native_hawaiian/pacific_islander',
       'percent_non-hispanic_white', 'recip_county', 'recip_state',
       'series_complete_pop_pct', 'series_complete_yes',
       'series_complete_12plus', 'series_complete_12pluspop_pct',
       'series_complete_18plus', 'series_complete_18pluspop_pct',
       'series_complete_65plus', 'series_complete_65pluspop_pct',
       'completeness_pct', 'administered_dose1_recip',
       'administered_dose1_pop_pct', 'administered_dose1_rec

In [9]:
df['series_complete_yes']

0        14892
1        27653
2         5151
3         7123
4         4913
         ...  
3111    563572
3112     18474
3113     19927
3114      9871
3115      7906
Name: series_complete_yes, Length: 3116, dtype: int64

In [6]:
pruned = df[['state','social_vulnerability_index_(svi)','estimated_hesitant_or_unsure',
           'percent_hispanic','percent_non-hispanic_american_indian/alaska_native',
           'percent_non-hispanic_asian','percent_non-hispanic_black','percent_non-hispanic_native_hawaiian/pacific_islander',
           'percent_non-hispanic_white']]

In [6]:
X = pruned.drop(columns=['estimated_hesitant_or_unsure'])
y = pruned['estimated_hesitant_or_unsure']

In [7]:
X = pd.get_dummies(data=X, columns=['state'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   random_state=42)

# Predicting Vaccine Hestitation

## Linear Regression
(0.0023815017692421525, 0.0025046182379516026)

In [9]:
linreg=LinearRegression()
linreg.fit(X_train, y_train)
mean_squared_error(y_train, linreg.predict(X_train)),mean_squared_error(y_test, linreg.predict(X_test))

(0.00018690041483654678, 0.00022874974127649097)

In [10]:
coefs = pd.DataFrame(list(zip(X.columns,linreg.coef_)), columns=['variable','coefficient'])

In [11]:
coefs.sort_values(by='coefficient',ascending=False)

,variable,coefficient
5,percent_non-hispanic_native_hawaiian/pacific_i...,1.738410e-01
32,state_MONTANA,1.175782e-01
2,percent_non-hispanic_american_indian/alaska_na...,1.110447e-01
56,state_WYOMING,1.091615e-01
30,state_MISSISSIPPI,9.441496e-02
6,percent_non-hispanic_white,8.884735e-02
8,state_ALASKA,7.976118e-02
18,state_IDAHO,7.557790e-02
4,percent_non-hispanic_black,7.516749e-02
24,state_LOUISIANA,7.360075e-02


## Random Forests
(4.793140984201448e-05, 0.0003138931016051236)

In [12]:
RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [73]:
gs = GridSearchCV(estimator=RandomForestRegressor(),
                 param_grid={
                     'n_estimators':[500],
                     'max_depth':np.linspace(10,50,5),
                     'warm_start':[True,False],
                 }, cv=5, n_jobs=-1,
                 scoring='neg_mean_squared_error')

In [74]:
%%time

gs.fit(X_train,y_train)

Wall time: 1min 15s


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': array([10., 20., 30., 40., 50.]),
                         'n_estimators': [500], 'warm_start': [True, False]},
             scoring='neg_mean_squared_error')

In [75]:
mean_squared_error(y_train, gs.predict(X_train)),mean_squared_error(y_test, gs.predict(X_test))

(4.793140984201448e-05, 0.0003138931016051236)

In [76]:
gs.best_params_

{'max_depth': 40.0, 'n_estimators': 500, 'warm_start': False}

## AdaBoost
(0.001259134303402612, 0.00137422633741578)

In [33]:
AdaBoostRegressor().get_params()

{'base_estimator': None,
 'learning_rate': 1.0,
 'loss': 'linear',
 'n_estimators': 50,
 'random_state': None}

In [77]:
ada_gs = GridSearchCV(estimator=AdaBoostRegressor(),
                     param_grid={
                         'learning_rate': np.linspace(.1,2,5),
                         'loss': ['linear','square','exponential'],
                         'n_estimators': [500],
                         'random_state': [42]
                     },cv=5, n_jobs=-1,
                     scoring='neg_mean_squared_error')

In [78]:
%%time

ada_gs.fit(X_train, y_train)

Wall time: 34.5 s


GridSearchCV(cv=5, estimator=AdaBoostRegressor(), n_jobs=-1,
             param_grid={'learning_rate': array([0.1  , 0.575, 1.05 , 1.525, 2.   ]),
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [500], 'random_state': [42]},
             scoring='neg_mean_squared_error')

In [79]:
ada_gs.best_params_

{'learning_rate': 1.525,
 'loss': 'square',
 'n_estimators': 500,
 'random_state': 42}

In [80]:
mean_squared_error(y_train, ada_gs.predict(X_train)),mean_squared_error(y_test, ada_gs.predict(X_test))

(0.001259134303402612, 0.00137422633741578)

## XGBoost
(1.7443373963809892e-05, 0.0002301318735520251)

In [63]:
XGBRegressor().get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'enable_categorical': False,
 'gamma': None,
 'gpu_id': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [81]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [82]:
mean_squared_error(y_train, xgb.predict(X_train)),mean_squared_error(y_test, xgb.predict(X_test))

(3.855291674682709e-05, 0.00023472043586748872)

In [98]:
xgb_gs = GridSearchCV(estimator=XGBRegressor(),
                     param_grid={
                         'max_depth':[10,11,12,13],
                         'learning_rate':np.linspace(.01,.1,3),
                         'n_estimators':[200],
                         'random_state':[42]
                     },
                     scoring='neg_mean_squared_error',
                     n_jobs=-1,
                     cv=5)

In [99]:
xgb_gs.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n...None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                            

In [100]:
xgb_gs.best_params_

{'learning_rate': 0.1,
 'max_depth': 10,
 'n_estimators': 200,
 'random_state': 42}

In [101]:
mean_squared_error(y_train, xgb_gs.predict(X_train)),mean_squared_error(y_test, xgb_gs.predict(X_test))

(1.7443373963809892e-05, 0.0002301318735520251)

# Estimating Vaccine Administrations

In [8]:
df.columns

Index(['fips_code', 'state', 'estimated_hesitant',
       'estimated_hesitant_or_unsure', 'estimated_strongly_hesitant',
       'social_vulnerability_index_(svi)', 'svi_category',
       'cvac_level_of_concern_for_vaccination_rollout',
       'cvac_level_of_concern',
       'percent_adults_fully_vaccinated_against_covid-19_(as_of_6/10/21)',
       'percent_hispanic',
       'percent_non-hispanic_american_indian/alaska_native',
       'percent_non-hispanic_asian', 'percent_non-hispanic_black',
       'percent_non-hispanic_native_hawaiian/pacific_islander',
       'percent_non-hispanic_white', 'recip_county', 'recip_state',
       'series_complete_pop_pct', 'series_complete_yes',
       'series_complete_12plus', 'series_complete_12pluspop_pct',
       'series_complete_18plus', 'series_complete_18pluspop_pct',
       'series_complete_65plus', 'series_complete_65pluspop_pct',
       'completeness_pct', 'administered_dose1_recip',
       'administered_dose1_pop_pct', 'administered_dose1_rec

In [28]:
X = df[['estimated_hesitant','estimated_hesitant_or_unsure','estimated_strongly_hesitant','social_vulnerability_index_(svi)',
     'cvac_level_of_concern_for_vaccination_rollout','metro_status']]
y = df['administered_dose1_recip']

In [31]:
X = pd.get_dummies(columns=['metro_status'],data=X)

In [43]:
# 0 Value for Metro_Status

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   random_state=42)

## Linear Regression
(193961.9332486839, 346720.08059892996)

In [71]:
linreg=LinearRegression()
linreg.fit(X_train, y_train)
mean_squared_error(y_train, linreg.predict(X_train),squared=False),mean_squared_error(y_test, linreg.predict(X_test),squared=False)

(193961.9332486839, 346720.08059892996)

In [69]:
coefs = pd.DataFrame(list(zip(X.columns,linreg.coef_)), columns=['variable','coefficient'])

In [70]:
coefs.sort_values(by='coefficient',ascending=False)

,variable,coefficient
0,estimated_hesitant,1.028218e+06
3,social_vulnerability_index_(svi),1.417404e+05
2,estimated_strongly_hesitant,1.232175e+05
6,metro_status_Metro,7.776130e+04
5,metro_status_0,-2.434905e+04
4,cvac_level_of_concern_for_vaccination_rollout,-3.184069e+04
7,metro_status_Non-metro,-5.341225e+04
1,estimated_hesitant_or_unsure,-1.858342e+06


## Random Forests
(70054.81896835605, 323067.93239419203)

In [72]:
RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [85]:
np.linspace(10,50,5)

array([10., 20., 30., 40., 50.])

In [97]:
gs = GridSearchCV(estimator=RandomForestRegressor(),
                 param_grid={
                     'n_estimators':[500],
                     'max_depth':np.linspace(10,50,5),
                     'warm_start':[True,False],
                     'min_samples_leaf':[1,5,10]
                 }, cv=5, n_jobs=-1)

In [98]:
%%time

gs.fit(X_train,y_train)

Wall time: 1min 46s


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': array([10., 20., 30., 40., 50.]),
                         'min_samples_leaf': [1, 5, 10], 'n_estimators': [500],
                         'warm_start': [True, False]})

In [95]:
mean_squared_error(y_train, gs.predict(X_train)),mean_squared_error(y_test, gs.predict(X_test))

(25242655903.57451, 103248116763.35637)

In [96]:
gs.best_params_

{'max_depth': 30.0,
 'min_samples_leaf': 10,
 'n_estimators': 500,
 'warm_start': True}

## AdaBoost
(155936.03349672462, 351135.7272933748)

In [78]:
AdaBoostRegressor().get_params()

{'base_estimator': None,
 'learning_rate': 1.0,
 'loss': 'linear',
 'n_estimators': 50,
 'random_state': None}

In [79]:
ada_gs = GridSearchCV(estimator=AdaBoostRegressor(),
                     param_grid={
                         'learning_rate': np.linspace(.1,2,5),
                         'loss': ['linear','square','exponential'],
                         'n_estimators': [500],
                         'random_state': [42]
                     },cv=5, n_jobs=-1,
                     scoring='neg_mean_squared_error')

In [80]:
%%time

ada_gs.fit(X_train, y_train)

Wall time: 13.5 s


GridSearchCV(cv=5, estimator=AdaBoostRegressor(), n_jobs=-1,
             param_grid={'learning_rate': array([0.1  , 0.575, 1.05 , 1.525, 2.   ]),
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [500], 'random_state': [42]},
             scoring='neg_mean_squared_error')

In [81]:
ada_gs.best_params_

{'learning_rate': 1.525,
 'loss': 'square',
 'n_estimators': 500,
 'random_state': 42}

In [84]:
mean_squared_error(y_train, ada_gs.predict(X_train),squared=False),mean_squared_error(y_test, ada_gs.predict(X_test),squared=False)

(155936.03349672462, 351135.7272933748)

## XGB
(22341.454376442274, 336950.1166971789)

In [53]:
XGBRegressor().get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'enable_categorical': False,
 'gamma': None,
 'gpu_id': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [54]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [62]:
mean_squared_error(y_train, xgb.predict(X_train)),mean_squared_error(y_test, xgb.predict(X_test))

(232573256.65102306, 113693461838.35988)

In [64]:
xgb_gs = GridSearchCV(estimator=XGBRegressor(),
                     param_grid={
                         'max_depth':[10,11,12,13],
                         'learning_rate':np.linspace(.01,.1,3),
                         'n_estimators':[500],
                         'random_state':[42]
                     },
                     scoring='neg_mean_squared_error',
                     n_jobs=-1,
                     cv=5)

Wall time: 0 ns


In [65]:
%%time

xgb_gs.fit(X_train,y_train)

Wall time: 1min 16s


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n...None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                            

In [66]:
xgb_gs.best_params_

{'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 500,
 'random_state': 42}

In [67]:
mean_squared_error(y_train, xgb_gs.predict(X_train),squared=False),mean_squared_error(y_test, xgb_gs.predict(X_test),squared=False)

(22341.454376442274, 336950.1166971789)